In [387]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
from fake_useragent import UserAgent as ua
import random

In [388]:
chromedriver = "/Applications/chromedriver" 
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
# path_to_extension = '/Library/Application Support/Google/Chrome/Default/Extensions/cfhdojbkjhnklbpkdaibdccddilifddb/3.10.2_0'
# #add adblock extention to load my selenium with adblock

# options = webdriver.ChromeOptions()
# options.add_argument(path_to_extension)



In [389]:
ua = UserAgent()
full_header = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
   'user-agent': ua.random
}

In [400]:
driver = webdriver.Chrome(executable_path= chromedriver)

In [6]:
# #print(nonprofit_link)
# print(nonprofit_name)
# print(nonprofit_type) ##FIGURE THIS PART OUT LATER

In [401]:
page = 'https://projects.propublica.org/nonprofits/search?c_code%5Bid%5D=&ntee%5Bid%5D=&page=8&q=New+York%2C+NY&state%5Bid%5D=NY'
driver.get(page)

In [402]:
single_nonprofit= []
valid_nonprofit= []
nonprofit_link = []
nonprofit_name = []
nonprofit_type = []
nonprofit_data = []
#page = 'https://projects.propublica.org/nonprofits/search?c_code%5Bid%5D=&ntee%5Bid%5D=&page=94&q=united+states&state%5Bid%5D='
#page = "https://projects.propublica.org/nonprofits/search?utf8=%E2%9C%93&q=united+states&state%5Bid%5D=&ntee%5Bid%5D=&c_code%5Bid%5D="


for i in range(0,101):
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find('table')
        rows = [row for row in table.find_all('tr')] 

        for row in enumerate(rows):
            if row[0] != 0:
                if(len((row[1].find('td', class_ = 'numeric').text))) > 3:
                       valid_nonprofit.append(row[1])

        for nonprofit in valid_nonprofit:
            nonprofit_link.append('https://projects.propublica.org' + nonprofit.find('a')['href'])
            nonprofit_name.append(nonprofit.find('a').text)
            nonprofit_type.append(nonprofit.find_all('td')[2].text)

        for link in nonprofit_link:
            driver.switch_to.window(driver.window_handles[0])
            driver.execute_script(f'''window.open('{link}', '_blank');''')
            time.sleep(.5+2*random.random())
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(.5+2*random.random())
            try:
                time.sleep(2.5+2*random.random())
                full_filing = driver.find_element_by_partial_link_text("Full Filing")
                full_filing.click()
                response = requests.get(driver.current_url, headers= full_header)
                page= response.text
                soup = BeautifulSoup(page, 'html.parser')
                new_link = soup.find('iframe').attrs['src']
                document_link = 'https://projects.propublica.org' + new_link
                document_link
                url = document_link
                response = requests.get(url, headers = full_header)
                page= response.text
                soup = BeautifulSoup(page, 'html.parser')
                doc_htmltxt = soup.find('iframe').attrs['srcdoc']
                docsoup = BeautifulSoup(doc_htmltxt, 'html.parser')
                doc_htmltxt2 = soup.find_all('iframe')[1].attrs['srcdoc']
                docsoup2 = BeautifulSoup(doc_htmltxt2, 'html.parser')
                time.sleep(.5+2*random.random())
                if docsoup.find('span', class_ = 'styFormNumber').text != '990':
                    driver.close()
                    continue
                else:
                    if docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/ReturnHeader[1]/Filer[1]/BusinessName[1]/BusinessNameLine1Txt[1]') is not None:
                        single_nonprofit.append(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/ReturnHeader[1]/Filer[1]/BusinessName[1]/BusinessNameLine1Txt[1]').text)
                    else: 
                        driver.close()
                        continue
                    single_nonprofit.append(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/ReturnHeader[1]/Filer[1]/USAddress[1]/StateAbbreviationCd[1]').text)
                    single_nonprofit.append(int(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/VotingMembersGoverningBodyCnt[1]').text.replace(',','')))
                    single_nonprofit.append(int('20' + docsoup.find('span', class_ = 'styTYColor').text))
                    single_nonprofit.append(int(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalEmployeeCnt[1]').text.replace(',','')))
                    single_nonprofit.append(float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/CYTotalRevenueAmt[1]').text.replace(',', '')))
                    single_nonprofit.append(float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/CYTotalExpensesAmt[1]').text.replace(',', '')))
                    single_nonprofit.append(float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalAssetsEOYAmt[1]').text.replace(',', '')))
                    single_nonprofit.append(float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalLiabilitiesEOYAmt[1]').text.replace(',', '')))
                    if len(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/CYSalariesCompEmpBnftPaidAmt[1]'))>2:
                        single_nonprofit.append(float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/CYSalariesCompEmpBnftPaidAmt[1]').text.replace(',', '')))
                    else:  single_nonprofit.append(0) 
                    if len(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/FSAuditedInd[1]').text)>4:
                        single_nonprofit.append('No')
                    else:
                        single_nonprofit.append(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/FSAuditedInd[1]').text)
                    if docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalReportableCompFromOrgAmt[1]') is not None:
                         single_nonprofit.append(float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalReportableCompFromOrgAmt[1]').text.replace(',', '')))
                    else:  single_nonprofit.append(0)
                    if docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotReportableCompRltdOrgAmt[1]') is not None:
                        single_nonprofit.append(float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotReportableCompRltdOrgAmt[1]').text.replace(',', '')))
                    else:  single_nonprofit.append(0)
                    if docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalOtherCompensationAmt[1]') is not None:
                         single_nonprofit.append(float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalOtherCompensationAmt[1]').text.replace(',', '')))
                    else:  single_nonprofit.append(0)
                    if docsoup2.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990ScheduleA[1]/PublicSupportCY509Pct[1]') is not None:
                         single_nonprofit.append(float(docsoup2.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990ScheduleA[1]/PublicSupportCY509Pct[1]').text.replace('%','')))
                    else:  single_nonprofit.append(0) 
                    if docsoup2.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990ScheduleA[1]/InvestmentIncomeCYPct[1]') is not None:
                        single_nonprofit.append(float(docsoup2.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990ScheduleA[1]/InvestmentIncomeCYPct[1]').text.replace('%','')))
                    else:  single_nonprofit.append(0)
                driver.close()


            except NoSuchElementException:
                print('No Such')
                time.sleep(3+2*random.random())
                single_nonprofit.clear()
                driver.close()
                continue
            except ElementClickInterceptedException:
                print('Intercept')
                time.sleep(1.5+2*random.random())
                close_button = driver.find_element_by_class_name("close-btn")
                driver.execute_script("arguments[0].click();", close_button)
                time.sleep(.5+2*random.random())
                single_nonprofit.clear()
                #driver.close()
            if len(nonprofit_data) == 0:
                    nonprofit_data = single_nonprofit
            elif len(nonprofit_data)!= 0 and len(single_nonprofit)>2:
                    nonprofit_data = np.vstack([nonprofit_data, single_nonprofit])
            else:
                pass

            print(single_nonprofit)
            single_nonprofit.clear()
                
        driver.switch_to.window(driver.window_handles[0])
        driver.refresh()
        time.sleep(.1 + 2*random.random())
        next_page_btn = driver.find_element_by_partial_link_text("Next")
        next_page_btn.click()
        time.sleep(8 + 5*random.random())
        valid_nonprofit.clear()
        nonprofit_link.clear()

    except NoSuchElementException:
        time.sleep(1+2*random.random())
        close_button = driver.find_element_by_class_name("close-btn")
        driver.execute_script("arguments[0].click();", close_button)
        time.sleep(1.5*random.random())
        next_page_btn = driver.find_element_by_partial_link_text("Next")
        next_page_btn.click
        time.sleep(8 + 5*random.random())
        valid_nonprofit.clear()
        nonprofit_link.clear()
    except ElementClickInterceptedException:
        time.sleep(1.5+ 2*random.random())
        close_button = driver.find_element_by_class_name("close-btn")
        driver.execute_script("arguments[0].click();", close_button)
        time.sleep(1 + 2*random.random())
        next_page_btn = driver.find_element_by_partial_link_text("Next")
        next_page_btn.click
        time.sleep(11 + 2*random.random())
        valid_nonprofit.clear()
        nonprofit_link.clear()

['SEAPORT MUSEUM NEW YORK', 'NY', 7, 2018, 108, 4223224.0, 5356985.0, 14852203.0, 607477.0, 0, 'Yes', 197981.0, 0.0, 50171.0, 0, 0]
['NEW YORK FOUNDLING', 'NY', 25, 2018, 2656, 157551022.0, 156458569.0, 151287909.0, 67995525.0, 0, 'Yes', 3300044.0, 0.0, 454051.0, 0, 0]
['APPNA NEW YORK CHAPTER INC', 'NY', 5, 2019, 0, 77373.0, 57512.0, 128207.0, 0.0, 0, 'No', 0.0, 0.0, 0.0, 100.0, 0.0]
['ARTS EAST NEW YORK INC', 'NY', 7, 2015, 0, 390472.0, 356888.0, 99278.0, 0.0, 0, 'No', 0, 0, 0, 0.0, 0.0]
['CITY ACCESS NEW YORK INC', 'NY', 8, 2018, 41, 3100071.0, 3131090.0, 1652680.0, 228581.0, 0, 'Yes', 134042.0, 0.0, 0.0, 99.9, 0.1]
['EDIBLE SCHOOLYARD NYC', 'NY', 20, 2018, 37, 2325191.0, 2130602.0, 1239253.0, 15023.0, 0, 'Yes', 368301.0, 0.0, 29645.0, 84.96, 0.02]
['NEW YORK ATHLETIC CLUB', 'NY', 19, 2018, 672, 61630228.0, 60785620.0, 81733603.0, 8115617.0, 0, 'Yes', 1869539.0, 0.0, 139674.0, 0, 0]
No Such
['NEW YORK CITY H2O', 'NY', 7, 2018, 0, 251960.0, 237267.0, 31496.0, 0.0, 0, 'No', 60000.0, 0

KeyboardInterrupt: 

In [ ]:
close_button = driver.find_element_by_class_name("close-btn")
driver.execute_script("arguments[0].click();", close_button)

In [ ]:
len(range(0,3))

In [ ]:
next_page_btn = driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/nav[1]/span[9]/a')
next_page_btn.click()
time.sleep(4)
valid_nonprofit.clear()
nonprofit_link.clear()
next_page_btn

In [116]:
single_nonprofit
len(single_nonprofit)

0

In [403]:
print(len(nonprofit_data), nonprofit_data)

10 [['NEW YORK FOUNDLING' 'NY' '25' '2018' '2656' '157551022.0'
  '156458569.0' '151287909.0' '67995525.0' '0' 'Yes' '3300044.0' '0.0'
  '454051.0' '0' '0']
 ['NEW YORK FOUNDLING' 'NY' '25' '2018' '2656' '157551022.0'
  '156458569.0' '151287909.0' '67995525.0' '0' 'Yes' '3300044.0' '0.0'
  '454051.0' '0' '0']
 ['APPNA NEW YORK CHAPTER INC' 'NY' '5' '2019' '0' '77373.0' '57512.0'
  '128207.0' '0.0' '0' 'No' '0.0' '0.0' '0.0' '100.0' '0.0']
 ['ARTS EAST NEW YORK INC' 'NY' '7' '2015' '0' '390472.0' '356888.0'
  '99278.0' '0.0' '0' 'No' '0' '0' '0' '0.0' '0.0']
 ['CITY ACCESS NEW YORK INC' 'NY' '8' '2018' '41' '3100071.0' '3131090.0'
  '1652680.0' '228581.0' '0' 'Yes' '134042.0' '0.0' '0.0' '99.9' '0.1']
 ['EDIBLE SCHOOLYARD NYC' 'NY' '20' '2018' '37' '2325191.0' '2130602.0'
  '1239253.0' '15023.0' '0' 'Yes' '368301.0' '0.0' '29645.0' '84.96'
  '0.02']
 ['NEW YORK ATHLETIC CLUB' 'NY' '19' '2018' '672' '61630228.0'
  '60785620.0' '81733603.0' '8115617.0' '0' 'Yes' '1869539.0' '0.0'
  '13967

In [404]:
nonprofit_data_safe = np.vstack([nonprofit_data_safe, nonprofit_data])

In [405]:
print(len(nonprofit_data_safe), nonprofit_data_safe)

2173 [['UNITED STATES AIKIDO FEDERATION' 'CT' '7' ... '0' '99.75' '0.25']
 ['UNITED STATES AIKIDO FEDERATION' 'CT' '7' ... '0' '99.75' '0.25']
 ['Mercy Hospital of Devils Lake Foundation' 'ND' '7' ... '82772.0' '0'
  '0']
 ...
 ['NEW YORK CITY H2O' 'NY' '7' ... '0.0' '100.0' '0.0']
 ['NEW YORK FELLOWSHIP INC' 'NY' '7' ... '66935.0' '0' '0']
 ['NEW YORK FOLKLORE SOCIETY INC' 'NY' '10' ... '0' '0' '0']]


In [406]:
np.save('data_safe_pg_new', nonprofit_data_safe)

In [ ]:
# full_header = {
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
#     'accept-encoding': 'gzip, deflate, br',
#     'accept-language': 'en-US,en;q=0.8',
#     'upgrade-insecure-requests': '1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
# }

#AVOID CAPTCHA use response= requests.get(page_link, headers= full_header)

In [ ]:
url = 'https://projects.propublica.org/nonprofits/organizations/264381874/201933159349301658/full'
response = requests.get(url)
page= response.text
soup = BeautifulSoup(page, 'html.parser')

In [ ]:
soup.prettify()

In [ ]:
new_link = soup.find('iframe').attrs['src']
new_link

In [ ]:
document_link = 'https://projects.propublica.org' + new_link
document_link
url = document_link
response = requests.get(url)
page= response.text
soup = BeautifulSoup(page, 'html.parser')

In [ ]:
doc_htmltxt = soup.find('iframe').attrs['srcdoc']
docsoup = BeautifulSoup(doc_htmltxt, 'html.parser')
doc_htmltxt2 = soup.find_all('iframe')[1].attrs['srcdoc']
docsoup2 = BeautifulSoup(doc_htmltxt2, 'html.parser')

In [ ]:
docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/ReturnHeader[1]/Filer[1]/BusinessName[1]/BusinessNameLine1Txt[1]').text
docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/ReturnHeader[1]/Filer[1]/USAddress[1]/StateAbbreviationCd[1]').text
int(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/VotingMembersGoverningBodyCnt[1]').text)
int('20' + docsoup.find('span', class_ = 'styTYColor').text)
docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalEmployeeCnt[1]').text
float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/CYTotalRevenueAmt[1]').text.replace(',', ''))
float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/CYTotalExpensesAmt[1]').text.replace(',', ''))
float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalAssetsEOYAmt[1]').text.replace(',', ''))
float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalLiabilitiesEOYAmt[1]').text.replace(',', ''))
if len(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/CYSalariesCompEmpBnftPaidAmt[1]'))>2:
    float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/CYSalariesCompEmpBnftPaidAmt[1]').text.replace(',', ''))
else: 0    
docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/FSAuditedInd[1]').text

if docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalReportableCompFromOrgAmt[1]') is not None:
    float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalReportableCompFromOrgAmt[1]').text.replace(',', ''))
else: 0

if docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotReportableCompRltdOrgAmt[1]') is not None:
    float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotReportableCompRltdOrgAmt[1]').text.replace(',', ''))
else: 0
if docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalOtherCompensationAmt[1]') is not None:
    float(docsoup.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990[1]/TotalOtherCompensationAmt[1]').text.replace(',', ''))
else: 0

if docsoup2.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990ScheduleA[1]/PublicSupportCY509Pct[1]') is not None:
    float(docsoup2.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990ScheduleA[1]/PublicSupportCY509Pct[1]').text.replace('%',''))
else: 0 

if docsoup2.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990ScheduleA[1]/InvestmentIncomeCYPct[1]') is not None:
    float(docsoup2.find('span', id = '/AppData/SubmissionHeaderAndDocument/SubmissionDocument/IRS990ScheduleA[1]/InvestmentIncomeCYPct[1]').text.replace('%',''))
else: 0     
# 
# docsoup.find('span', id = '').text
# docsoup2.find('span', id = '').text
#Name, State, num governing, filing year, num employees, total revenue, total expenses, total assets, total liabilities, salaries paid, independent audit?, 
#Exec Comp from org, Exec comp from related, Exec comp other, public support %, investment income %

